### Install Required Packages

In [5]:
!cat requirements.txt | xargs -n 1 -L 1 pip install --upgrade
!conda install -c conda-forge -y ta-lib

  Using cached numpy-1.23.5.tar.gz (10.7 MB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... error
  error: subprocess-exited-with-error
  
  × Getting requirements to build wheel did not run successfully.
  │ exit code: 1
  ╰─> [33 lines of output]
      Traceback (most recent call last):
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 353, in <module>
          main()
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 335, in main
          json_out['return_val'] = hook(**hook_input['kwargs'])
                                   ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
        File "/opt/anaconda3/lib/python3.12/site-packages/pip/_vendor/pyproject_hooks/_in_process/_in_process.py", line 112, in get_requires_for_build_wheel
          backend = _build_backend()
                    ^^^^^^^^^^^^^^^^
        File "/opt/anaconda3

### Import Required Package

In [13]:
import pandas as pd
import talib

import json

### Import Tidal

In [14]:
import tidal as td

### Initialize Plumber

In [15]:
from pathlib import Path
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

PLUMBER_HOST = "https://dev-api.ddt-dst.cc/api/plumber/"
with open(f'{str(Path.home())}/.config/gcloud/application_default_credentials.json') as plumber_token:
    token = json.load(plumber_token)

### Load market data and compute technical indicators

In [20]:
DEFAULT_STOCKS = ["2330", "2204", "2455", "1303", "2603", "2605", "2317", "6222", "1216", "2850", "6486", "3443", "3231", "1473", "2023", "1459", "2618", "1603", "2454", "2303"]
start_date = "2019-01-01"
end_date = "2020-12-31"

quote_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": ",".join([stock for stock in DEFAULT_STOCKS]),
    },
)
quote_data.index = quote_data.index.set_levels(
    pd.to_datetime(quote_data.index.levels[1]),
    level=1,
)
quote_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

instruments = quote_data.index.get_level_values('instrument').unique()

new_db = {}

for instrument in instruments:
    new_db[instrument] = quote_data.loc[instrument].assign(MACD=talib.MACD(quote_data.loc[instrument].close, fastperiod=12, slowperiod=26, signalperiod=9)[2])

quote_data = pd.concat(new_db, names=["instrument"], sort=False).dropna()

quote_data

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/formatters.py:347: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/anaconda3/lib/python3.12/site-packages/IPython/core/formatters.py:347: FutureWarning: MultiIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/anaconda3/lib/python3.12/site-packages/IPython/core/formatters.py:347: FutureWarning: DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


open      high       low     close     volume   
instrument datetime                                                        
2330       2019-02-27  204.7903  205.6490  203.5023  205.2197   37947889  \
           2019-03-04  205.6490  205.6490  201.3557  202.2144   45164513   
           2019-03-05  200.9264  201.3557  199.6384  200.0677   23708407   
           2019-03-06  201.7850  202.2144  200.4970  200.9264   20811226   
           2019-03-07  201.7850  201.7850  200.0677  200.9264   17226192   
...                         ...       ...       ...       ...        ...   
2303       2020-12-25   37.8263   38.1074   37.3846   37.5453   97512133   
           2020-12-28   37.8263   39.2318   37.7460   39.0310  221527717   
           2020-12-29   39.5129   39.7136   37.9067   38.0673  251092444   
           2020-12-30   38.1074   38.8302   37.8263   38.7901  153219574   
           2020-12-31   39.1916   39.2318   36.8626   37.8665  302544317   

                           MACD  
instrument datetime              
2330       2019-02-27  0.620619  
           2019-03-04  0.297274  
           2019-03-05 -0.099059  
           2019-03-06 -0.328340  
           2019-03-07 -0.494949  
...                         ...  
2303       2020-12-25 -0.407249  
           2020-12-28 -0.306262  
           2020-12-29 -0.309516  
           2020-12-30 -0.269601  
           2020-12-31 -0.309177  

[8879 rows x 6 columns]

In [21]:
quote_data

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/formatters.py:347: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/anaconda3/lib/python3.12/site-packages/IPython/core/formatters.py:347: FutureWarning: MultiIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/anaconda3/lib/python3.12/site-packages/IPython/core/formatters.py:347: FutureWarning: DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


open      high       low     close     volume   
instrument datetime                                                        
2330       2019-02-27  204.7903  205.6490  203.5023  205.2197   37947889  \
           2019-03-04  205.6490  205.6490  201.3557  202.2144   45164513   
           2019-03-05  200.9264  201.3557  199.6384  200.0677   23708407   
           2019-03-06  201.7850  202.2144  200.4970  200.9264   20811226   
           2019-03-07  201.7850  201.7850  200.0677  200.9264   17226192   
...                         ...       ...       ...       ...        ...   
2303       2020-12-25   37.8263   38.1074   37.3846   37.5453   97512133   
           2020-12-28   37.8263   39.2318   37.7460   39.0310  221527717   
           2020-12-29   39.5129   39.7136   37.9067   38.0673  251092444   
           2020-12-30   38.1074   38.8302   37.8263   38.7901  153219574   
           2020-12-31   39.1916   39.2318   36.8626   37.8665  302544317   

                           MACD  
instrument datetime              
2330       2019-02-27  0.620619  
           2019-03-04  0.297274  
           2019-03-05 -0.099059  
           2019-03-06 -0.328340  
           2019-03-07 -0.494949  
...                         ...  
2303       2020-12-25 -0.407249  
           2020-12-28 -0.306262  
           2020-12-29 -0.309516  
           2020-12-30 -0.269601  
           2020-12-31 -0.309177  

[8879 rows x 6 columns]

### Load benchmark data

In [23]:
benchmark_inst = "0050"
benchmark_data = pd.read_parquet(
    f"{PLUMBER_HOST}stocks/tw/ohlcv",
    storage_options={
        "gcp-token": json.dumps(token),
        "start-date": start_date,
        "end-date": end_date,
        "tickers": benchmark_inst,
    },
)
benchmark_data.index = benchmark_data.index.set_levels(
    pd.to_datetime(benchmark_data.index.levels[1]),
    level=1,
)
benchmark_data.rename_axis(index={
    'ticker': 'instrument'
}, inplace=True)

benchmark_data = benchmark_data.loc[benchmark_inst][['close']]
benchmark_data

/opt/anaconda3/lib/python3.12/site-packages/IPython/core/formatters.py:347: FutureWarning: Index.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()
/opt/anaconda3/lib/python3.12/site-packages/IPython/core/formatters.py:347: FutureWarning: DatetimeIndex.format is deprecated and will be removed in a future version. Convert using index.astype(str) or index.map(formatter) instead.
  return method()


,close
datetime,
2019-01-02,60.3448
2019-01-03,59.7743
2019-01-04,58.8372
2019-01-07,60.2633
2019-01-08,60.0188
...,...
2020-12-25,104.8235
2020-12-28,105.7488
2020-12-29,105.6607


### Initialize Tidal
1. Initialize Tidal object
2. Add Quote data (pd.DataFrame)
3. Set strategy object (td.BaseStrategy)
4. Add metric objects (td.BaseMetic)

In [24]:
td.__version__

'1.1.42'

In [25]:
# Initialize Tidal object
tidal = td.Tidal(init_cash=10000000, slip_ticks=0, stock_config=td.StockConfig.TW, reqMem="2248Mi", ignore_volume_size=True, handle_zero_volume=False)
# Add quote data
tidal.add_quote(quote_data)

# Set strategy object
tidal.set_strategy(td.strategy.TopkDropout(5, 2, 'MACD'))

# Set metric objects
tidal.add_metric(td.metric.AccountInfo())
tidal.add_metric(td.metric.AdditionalInfo())
tidal.add_metric(td.metric.PositionInfo())
tidal.add_metric(td.metric.Portfolio(benchmark_data))

KeyboardInterrupt: 

### Stock Config

In [ ]:
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.0004275, Min Commission:20.0, Transaction Tax:0.003}

### Instrument Configs

In [ ]:
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0}}

### Config Modification

In [ ]:
# Set commission to 77% off
tidal.exchange.set_stock_config(commission=0.001425 * 0.23)
tidal.exchange.stock_config

InstConfig {Margin:0.0, Tick Size:0.002, Tick Value:0.002, Trade Unit:1000, Commission:0.00032775, Min Commission:20.0, Transaction Tax:0.003}

### Add New Instrument Config

In [ ]:
# Add MGC config
tidal.exchange.set_config(instrument='MGC', margin=787., tick_size=0.1, tick_value=1., trade_unit=1, commission=2., min_commission=0., transaction_tax=0.)
tidal.exchange.inst_configs

{'MTX': InstConfig {Margin:46000.0, Tick Size:1.0, Tick Value:50.0, Trade Unit:1, Commission:20.0, Min Commission:0.0, Transaction Tax:2e-05},
 'TX': InstConfig {Margin:184000.0, Tick Size:1.0, Tick Value:200.0, Trade Unit:1, Commission:40.0, Min Commission:0.0, Transaction Tax:2e-05},
 'NQ': InstConfig {Margin:16500.0, Tick Size:0.25, Tick Value:5.0, Trade Unit:1, Commission:1.85, Min Commission:0.0, Transaction Tax:0.0},
 'MGC': InstConfig {Margin:787.0, Tick Size:0.1, Tick Value:1.0, Trade Unit:1, Commission:2.0, Min Commission:0.0, Transaction Tax:0.0}}

### Start Backtesting

In [ ]:
tidal.backtest()

Tidal Backtesting:   5%|▌         | 23/454 [00:02<00:55,  7.72it/s, cash=2.6e+5, pnl=-2.61e+4, position_cost=9.52e+6, value=9.75e+6] 

Tidal Backtesting: 100%|██████████| 454/454 [00:39<00:00, 11.50it/s, cash=1.31e+5, pnl=1.47e+6, position_cost=1e+7, value=1.16e+7]    


### Metric - AccountInfo

In [ ]:
tidal.metrics['AccountInfo'].report

,cash,trade_cost,position_cost,slip_cost,pnl,value,max_drawdown
datetime,,,,,,,
2019-02-27,1.000000e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+07,0.000000
2019-03-04,1.000000e+07,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+07,0.000000
2019-03-05,7.237681e+05,3.039289e+03,9.273193e+06,-5.152057e+03,-7.517114e+04,9.921790e+06,-0.007821
2019-03-06,1.275952e+05,9.912974e+03,9.862975e+06,-1.225006e+04,-4.569315e+04,9.944877e+06,-0.005512
2019-03-07,2.968355e+04,1.874163e+04,9.942410e+06,8.030852e+04,-2.176127e+05,9.754481e+06,-0.024552
...,...,...,...,...,...,...,...
2020-12-25,1.730890e+04,1.672832e+06,1.025058e+07,2.243912e+06,6.819387e+05,1.094982e+07,0.000000
2020-12-28,1.730890e+04,1.672832e+06,1.025058e+07,2.243912e+06,1.007756e+06,1.127564e+07,0.000000
2020-12-29,3.573191e+05,1.678157e+06,9.876658e+06,2.222475e+06,1.216278e+06,1.145026e+07,0.000000


### Metric - PositionInfo

In [ ]:
tidal.metrics['PositionInfo'].report

quantity      price  commission   slip_cost  \
instrument datetime                                                  
1216       2019-05-03   27000.0  61.109700  540.775013   2235.6000   
           2019-05-06   27000.0  61.109700  540.775013   2235.6000   
           2019-05-07   27000.0  61.109700  540.775013   2235.6000   
           2019-05-08   27000.0  61.109700  540.775013   2235.6000   
           2019-05-09   27000.0  61.109700  540.775013   2235.6000   
...                         ...        ...         ...         ...   
6486       2020-12-11   26000.0  70.003928  596.538476  50679.9384   
           2020-12-14   26000.0  70.003928  596.538476  50679.9384   
           2020-12-15   26000.0  70.003928  596.538476  50679.9384   
           2020-12-30   35000.0  72.447806  831.066899   5061.2240   
           2020-12-31   35000.0  72.447806  831.066899   5061.2240   

                               pnl  
instrument datetime                 
1216       2019-05-03       0.0000  
           2019-05-06  -15651.9000  
           2019-05-07    4471.2000  
           2019-05-08   58128.3000  
           2019-05-09   60363.9000  
...                            ...  
6486       2020-12-11   20913.6616  
           2020-12-14   -3632.9384  
           2020-12-15 -118186.3384  
           2020-12-30    3198.7760  
           2020-12-31   30733.2760  

[2177 rows x 5 columns]

In [ ]:
pi_report = tidal.metrics['PositionInfo'].report
pi_report.iloc[pi_report.index.get_level_values('datetime') == '2022-06-14']

,,quantity,price,commission,slip_cost,pnl
instrument,datetime,,,,,


### Metric - AdditionalInfo

In [ ]:
tidal.metrics['AdditionalInfo'].report

MACD
instrument datetime            
1216       2019-02-27  0.040105
           2019-03-04 -0.095713
           2019-03-05 -0.177846
           2019-03-06 -0.189909
           2019-03-07 -0.204557
...                         ...
6486       2020-12-25 -0.196322
           2020-12-28  0.092305
           2020-12-29  0.237339
           2020-12-30  0.314350
           2020-12-31  0.379117

[8879 rows x 1 columns]

### Trade Report

In [ ]:
tidal.trade_report

,win_num,lose_num,trade_num,pos_num,win_rate,profit,loss,trade_cost,pnl
instrument,,,,,,,,,
2303,7.0,3.0,10.0,0.0,0.700000,1.627643e+06,-9.198999e+04,7.011159e+04,1.535653e+06
2330,9.0,6.0,15.0,0.0,0.600000,1.253880e+06,-2.759367e+05,9.750049e+04,9.779433e+05
2603,9.0,11.0,20.0,1.0,0.450000,1.776056e+06,-8.070776e+05,2.717152e+06,9.689780e+05
2455,8.0,8.0,16.0,0.0,0.500000,1.062702e+06,-7.004614e+05,1.067074e+05,3.622402e+05
3231,9.0,7.0,16.0,0.0,0.562500,5.975250e+05,-2.508009e+05,1.038286e+05,3.467241e+05
2317,7.0,9.0,16.0,0.0,0.437500,7.032317e+05,-4.691996e+05,1.052570e+05,2.340321e+05
1216,3.0,8.0,11.0,0.0,0.272727,3.334657e+05,-1.898143e+05,7.556202e+04,1.436514e+05
6486,8.0,10.0,18.0,1.0,0.444444,4.613830e+05,-4.747103e+05,2.647591e+06,-1.332730e+04
2850,4.0,6.0,10.0,0.0,0.400000,7.784960e+04,-1.074179e+05,6.647040e+04,-2.956831e+04


### Traded instruments

In [ ]:
tidal.account.trades.keys()

dict_keys(['2454', '2330', '2455', '3443', '1303', '2618', '2204', '2605', '6486', '1603', '2317', '2850', '3231', '1216', '1473', '2603', '2303', '2023', '1459', '6222'])

### Trading History

In [ ]:
tidal.account.trades['2330']

[Trade {Instrument:2330, Open {Datetime:2019-03-05 00:00:00, Price:203.0054004, Commission:598.8151798299001, Slip cost:-8041.496399999858}, Close {Datetime:2019-03-06 00:00:00, Price:203.059068, Commission:598.973485833, Slip cost:-11922.012000000024, Close type:Market}, Transaction tax:5482.594835999999, Trade cost:6680.3835016629, Quantity:9000, Position cost:1827048.6036, PnL:-6197.375101663054},
 Trade {Instrument:2330, Open {Datetime:2019-03-21 00:00:00, Price:210.3795192, Commission:620.5669867602, Slip cost:7675.372799999991}, Close {Datetime:2019-03-28 00:00:00, Price:207.811544, Commission:612.992101914, Slip cost:11540.303999999907, Close type:Market}, Transaction tax:5610.911688, Trade cost:6844.4707766742, Quantity:9000, Position cost:1893415.6728, PnL:-29956.247576674257},
 Trade {Instrument:2330, Open {Datetime:2019-04-09 00:00:00, Price:219.4887012, Commission:647.4367963647001, Slip cost:3942.9108000000497}, Close {Datetime:2019-04-30 00:00:00, Price:224.6610774, Commi

### Plot chart by using Plotly

In [ ]:
tidal.analyzer.inst_chart(instrument='2330', metric_name='AdditionalInfo', plot_type=td.PlotType.LINE, scale=1.0)

FigureWidget({
    'data': [{'close': array([206.9292, 203.8989, 201.7344, ..., 480.8733, 490.2107, 494.8794])…

### Tidal Dashboard

In [ ]:
tidal.tdboard()

 * Serving Flask app 'tidal.tdboard'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on all addresses (0.0.0.0)
 * Running on http://127.0.0.1:42407
 * Running on http://10.136.9.21:42407
INFO:werkzeug:Press CTRL+C to quit


INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:40:35] "GET / HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:40:37] "GET /static/js/main.ae1b7240.js HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:40:54] "GET /static/css/main.3192b1a2.css HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:41:01] "GET /api/quote/inst_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:41:01] "GET /api/metric/metric_list HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:41:01] "GET /api/trade/trade_report HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:41:02] "GET /images/Tidal_Logo_white.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:41:03] "GET /Tidal_Logo.png HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:41:13] "GET /static/media/primeicons.78172950b335ccdb94e2.woff2 HTTP/1.1" 200 -
INFO:werkzeug:10.0.10.82 - - [21/Jun/2024 15:41:13] "POST /api/metric/metric_reports HTTP/1.1" 200 -
INFO:werkzeug:10